# import vectors

In [ ]:
import sys
import os
import pandas as pd
import pickle
from scipy import sparse


# Load labels from pickle file
with open("../01_vectorization/vectorization_artifacts/tf_idf/labels.pkl", "rb") as f:
    labels_df = pd.DataFrame(pickle.load(f))

print(labels_df.head())

# Load TF-IDF vectors from NPZ file
X_tfidf = sparse.load_npz(
    "../01_vectorization/vectorization_artifacts/tf_idf/X_tfidf.npz"
)

print(labels_df.head())
print("\nTF-IDF matrix shape:", X_tfidf.shape)

                target_name
0          rec.sport.hockey
1  comp.sys.ibm.pc.hardware
2     talk.politics.mideast
3  comp.sys.ibm.pc.hardware
4     comp.sys.mac.hardware


FileNotFoundError: [Errno 2] No such file or directory: '../01_vectorization/vectorization_artifacts/tf_idf/X_tfidf.npz'